# Proceso de ETL para la tabla de hechos de la base de datos NBA

In [1]:
#Carga de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

Carga de los datos

In [2]:
#CARGA DEL DATASET
data = pd.read_csv("game.csv", sep=',')

Primer vistaso a las columnas

In [3]:
pd.options.display.max_columns = 60
pd.options.display.max_columns

60

In [4]:
data.head(5)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,video_available_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,NaN,NaN,NaN,NaN,NaN,16.0,29.0,0.552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,-2,0,1610612752,NYK,New York Knicks,NYK @ HUS,W,24.0,NaN,NaN,NaN,NaN,NaN,20.0,26.0,0.769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,59.0,0.339,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,56.0,5,0,1610610031,PIT,Pittsburgh Ironmen,PIT @ BOM,L,16.0,72.0,0.222,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,51.0,-5,0,Regular Season
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,6,0,1610612738,BOS,Boston Celtics,BOS @ PRO,L,21.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,-6,0,Regular Season
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,63.0,16,0,1610612752,NYK,New York Knicks,NYK @ CHS,L,16.0,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,47.0,-16,0,Regular Season
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,-17,0,1610610036,WAS,Washington Capitols,WAS @ DEF,W,18.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,17,0,Regular Season


Se comprueba que el dataset cargado tiene la cantidad de registros y columnas especificadas

In [5]:
np.shape(data)

(65698, 55)

Se procedera a establecer un filtrado por fechas. Esto es para utilizar las temporadas siguientes (2020/2021) (2021/2022) (2022/2023) como fijan los parámetros del proyecto.

Dado que el final de la temporada anterior se retrasó debido la pandemia COVID-19, la NBA aprobó una fecha excepcional de inicio para el 22 de diciembre de 2020, cuando lo habitual era comenzar en el mes de octubre.

In [6]:
# Filtra los datos en fechas mayores a 2021-01-01.
data = data[data["game_date"]>'2020-12-31']
data = data[data["game_date"]<'2023-6-13']
np.shape(data)

(3877, 55)

In [7]:
data.tail(5)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,video_available_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
65693,42022,1610612748,MIA,Miami Heat,42200403,2023-06-07 00:00:00,MIA vs. DEN,L,240,34.0,92.0,0.370,11.0,35.0,0.314,15.0,19.0,0.789,10.0,23.0,33.0,20.0,7.0,3.0,4.0,22.0,94.0,-15,1,1610612743,DEN,Denver Nuggets,DEN @ MIA,W,41.0,80.0,0.513,5.0,18.0,0.278,22.0,27.0,0.815,13.0,45.0,58.0,28.0,3.0,5.0,14.0,18.0,109.0,15,1,Playoffs
65694,42022,1610612748,MIA,Miami Heat,42200404,2023-06-09 00:00:00,MIA vs. DEN,L,240,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,8.0,29.0,37.0,23.0,2.0,3.0,15.0,19.0,95.0,-13,1,1610612743,DEN,Denver Nuggets,DEN @ MIA,W,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,0.762,5.0,29.0,34.0,26.0,11.0,7.0,8.0,18.0,108.0,13,1,Playoffs
65695,42022,1610612743,DEN,Denver Nuggets,42200405,2023-06-12 00:00:00,DEN vs. MIA,W,240,38.0,84.0,0.452,5.0,28.0,0.179,13.0,23.0,0.565,11.0,46.0,57.0,21.0,6.0,7.0,15.0,13.0,94.0,5,1,1610612748,MIA,Miami Heat,MIA @ DEN,L,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,0.875,11.0,33.0,44.0,18.0,9.0,7.0,8.0,21.0,89.0,-5,1,Playoffs
65696,32022,1610616834,LBN,Team LeBron,32200001,2023-02-19 00:00:00,LBN vs. GNS,L,221,79.0,132.0,0.598,17.0,60.0,0.283,0.0,0.0,NaN,13.0,32.0,45.0,49.0,7.0,2.0,10.0,5.0,175.0,-9,1,1610616833,GNS,Team Giannis,GNS @ LBN,W,76.0,123.0,0.618,29.0,66.0,0.439,3.0,4.0,0.750,10.0,36.0,46.0,43.0,8.0,1.0,12.0,2.0,184.0,9,1,All-Star
65697,32022,1610616834,LBN,Team LeBron,32200001,2023-02-19 00:00:00,LBN vs. GNS,L,221,79.0,132.0,0.598,17.0,60.0,0.283,0.0,0.0,NaN,13.0,32.0,45.0,49.0,7.0,2.0,10.0,5.0,175.0,-9,1,1610616833,GNS,Team Giannis,GNS @ LBN,W,76.0,123.0,0.618,29.0,66.0,0.439,3.0,4.0,0.750,10.0,36.0,46.0,43.0,8.0,1.0,12.0,2.0,184.0,9,1,All Star


### Se consultan valores nulos por primera vez.

In [8]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

                        Valores Nulos  Porcentaje Nulos
season_id                           0          0.000000
team_id_home                        0          0.000000
team_abbreviation_home              0          0.000000
team_name_home                      0          0.000000
game_id                             0          0.000000
game_date                           0          0.000000
matchup_home                        0          0.000000
wl_home                             0          0.000000
min                                 0          0.000000
fgm_home                            0          0.000000
fga_home                            0          0.000000
fg_pct_home                         0          0.000000
fg3m_home                           0          0.000000
fg3a_home                           0          0.000000
fg3_pct_home                        0          0.000000
ftm_home                            0          0.000000
fta_home                            0          0

Observamos que la integridad de nuestros datos es súper sólida debido a que tenemos 2 datos faltantes unicamente, los cuales estan alojados en la columna ft_pct_home.


Imprimimos esos datos para tratarlos de forma particular según su situación.

In [9]:
valores_faltantes = data[data['ft_pct_home'].isnull()]
print(valores_faltantes)

       season_id  team_id_home team_abbreviation_home team_name_home  \
65696      32022    1610616834                    LBN    Team LeBron   
65697      32022    1610616834                    LBN    Team LeBron   

        game_id            game_date matchup_home wl_home  min  fgm_home  \
65696  32200001  2023-02-19 00:00:00  LBN vs. GNS       L  221      79.0   
65697  32200001  2023-02-19 00:00:00  LBN vs. GNS       L  221      79.0   

       fga_home  fg_pct_home  fg3m_home  fg3a_home  fg3_pct_home  ftm_home  \
65696     132.0        0.598       17.0       60.0         0.283       0.0   
65697     132.0        0.598       17.0       60.0         0.283       0.0   

       fta_home  ft_pct_home  oreb_home  dreb_home  reb_home  ast_home  \
65696       0.0          NaN       13.0       32.0      45.0      49.0   
65697       0.0          NaN       13.0       32.0      45.0      49.0   

       stl_home  blk_home  tov_home  pf_home  pts_home  plus_minus_home  \
65696       7.0      

El 19 de febrero de 2023, durante el NBA All-Star Game 2023, el equipo dirigido por LeBron James (Team LeBron) jugó contra el equipo dirigido por Giannis Antetokounmpo (Team Giannis). En ese juego, la estadística de porcentaje de tiros libres (FT%) para el Team LeBron fue del 87.5%. Rellenaremos el dato faltante con el dato real. Este dato fue sacado de la página oficial de la NBA (nba.com)

Antes de rellenar el dato podemos observar que la fila completa está repetida, por ende procedemos a eliminar una.

In [10]:
data = data.drop(65697)

Ahora si rellenamos el dato faltante de la columna 'ft_pct_home' con su valor real.

In [11]:
data.loc[65696, 'ft_pct_home'] = 0.875

In [12]:
valor_f1 = data.loc[65696]
print(valor_f1)

season_id                               32022
team_id_home                       1610616834
team_abbreviation_home                    LBN
team_name_home                    Team LeBron
game_id                              32200001
game_date                 2023-02-19 00:00:00
matchup_home                      LBN vs. GNS
wl_home                                     L
min                                       221
fgm_home                                 79.0
fga_home                                132.0
fg_pct_home                             0.598
fg3m_home                                17.0
fg3a_home                                60.0
fg3_pct_home                            0.283
ftm_home                                  0.0
fta_home                                  0.0
ft_pct_home                             0.875
oreb_home                                13.0
dreb_home                                32.0
reb_home                                 45.0
ast_home                          

Controlamos que no existan más datos duplicados.

In [13]:
duplicados = data.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = data[duplicados]
print(datos_duplicados)

Empty DataFrame
Columns: [season_id, team_id_home, team_abbreviation_home, team_name_home, game_id, game_date, matchup_home, wl_home, min, fgm_home, fga_home, fg_pct_home, fg3m_home, fg3a_home, fg3_pct_home, ftm_home, fta_home, ft_pct_home, oreb_home, dreb_home, reb_home, ast_home, stl_home, blk_home, tov_home, pf_home, pts_home, plus_minus_home, video_available_home, team_id_away, team_abbreviation_away, team_name_away, matchup_away, wl_away, fgm_away, fga_away, fg_pct_away, fg3m_away, fg3a_away, fg3_pct_away, ftm_away, fta_away, ft_pct_away, oreb_away, dreb_away, reb_away, ast_away, stl_away, blk_away, tov_away, pf_away, pts_away, plus_minus_away, video_available_away, season_type]
Index: []


Controlamos nuevamente que no tengamos valores faltantes.

In [14]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

                        Valores Nulos  Porcentaje Nulos
season_id                           0               0.0
team_id_home                        0               0.0
team_abbreviation_home              0               0.0
team_name_home                      0               0.0
game_id                             0               0.0
game_date                           0               0.0
matchup_home                        0               0.0
wl_home                             0               0.0
min                                 0               0.0
fgm_home                            0               0.0
fga_home                            0               0.0
fg_pct_home                         0               0.0
fg3m_home                           0               0.0
fg3a_home                           0               0.0
fg3_pct_home                        0               0.0
ftm_home                            0               0.0
fta_home                            0           

Ahora que sabemos que el dataset esta correctamente filtrado y sin valores faltantes procedemos a revisar el formato de nuestras columnas antes de exportar el dataset con el ETL finalizado.

In [15]:
print(data.dtypes)

season_id                   int64
team_id_home                int64
team_abbreviation_home     object
team_name_home             object
game_id                     int64
game_date                  object
matchup_home               object
wl_home                    object
min                         int64
fgm_home                  float64
fga_home                  float64
fg_pct_home               float64
fg3m_home                 float64
fg3a_home                 float64
fg3_pct_home              float64
ftm_home                  float64
fta_home                  float64
ft_pct_home               float64
oreb_home                 float64
dreb_home                 float64
reb_home                  float64
ast_home                  float64
stl_home                  float64
blk_home                  float64
tov_home                  float64
pf_home                   float64
pts_home                  float64
plus_minus_home             int64
video_available_home        int64
team_id_away  

Observamos que la columna de fechas esta en formato 'object'.

La cambiamos a formato 'datetime'.

In [16]:
data['game_date'] = pd.to_datetime(data['game_date'], errors='coerce')

Revisamos nuevamente el formato para corroborar los cambios.

In [17]:
print(data['game_date'].dtypes)

datetime64[ns]


In [18]:
data.head(5)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,video_available_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
61772,22020,1610612764,WAS,Washington Wizards,22000063,2020-12-31,WAS vs. CHI,L,240,42.0,84.0,0.500,14.0,29.0,0.483,32.0,41.0,0.780,9.0,28.0,37.0,27.0,8.0,4.0,14.0,27.0,130.0,-3,1,1610612741,CHI,Chicago Bulls,CHI @ WAS,W,50.0,92.0,0.543,14.0,36.0,0.389,19.0,25.0,0.760,9.0,35.0,44.0,34.0,9.0,3.0,17.0,27.0,133.0,3,1,Regular Season
61773,22020,1610612762,UTA,Utah Jazz,22000068,2020-12-31,UTA vs. PHX,L,240,35.0,84.0,0.417,12.0,34.0,0.353,13.0,19.0,0.684,10.0,36.0,46.0,16.0,8.0,3.0,14.0,15.0,95.0,-11,1,1610612756,PHX,Phoenix Suns,PHX @ UTA,W,41.0,84.0,0.488,17.0,33.0,0.515,7.0,10.0,0.700,5.0,36.0,41.0,26.0,5.0,6.0,15.0,17.0,106.0,11,1,Regular Season
61774,22020,1610612745,HOU,Houston Rockets,22000065,2020-12-31,HOU vs. SAC,W,240,45.0,86.0,0.523,13.0,36.0,0.361,19.0,25.0,0.760,10.0,35.0,45.0,26.0,10.0,6.0,18.0,14.0,122.0,3,1,1610612758,SAC,Sacramento Kings,SAC @ HOU,L,45.0,96.0,0.469,11.0,33.0,0.333,18.0,24.0,0.750,14.0,31.0,45.0,23.0,11.0,5.0,15.0,21.0,119.0,-3,1,Regular Season
61775,22020,1610612753,ORL,Orlando Magic,22000064,2020-12-31,ORL vs. PHI,L,240,33.0,95.0,0.347,7.0,28.0,0.250,19.0,21.0,0.905,10.0,38.0,48.0,17.0,2.0,5.0,11.0,15.0,92.0,-24,1,1610612755,PHI,Philadelphia 76ers,PHI @ ORL,W,44.0,91.0,0.484,15.0,33.0,0.455,13.0,15.0,0.867,7.0,47.0,54.0,23.0,7.0,10.0,15.0,23.0,116.0,24,1,Regular Season
61776,22020,1610612760,OKC,Oklahoma City Thunder,22000067,2020-12-31,OKC vs. NOP,L,240,30.0,80.0,0.375,15.0,48.0,0.313,5.0,15.0,0.333,7.0,33.0,40.0,24.0,6.0,3.0,19.0,20.0,80.0,-33,1,1610612740,NOP,New Orleans Pelicans,NOP @ OKC,W,44.0,91.0,0.484,13.0,36.0,0.361,12.0,19.0,0.632,14.0,44.0,58.0,23.0,9.0,4.0,13.0,16.0,113.0,33,1,Regular Season


Le cambiamos el nombre a la columna, ya que min es un caracter reservado de SQL.

In [19]:
data = data.rename(columns={'min': 'game_minutes'})

In [20]:
lista_a_dropear = ['video_available_away','video_available_home']

Eliminamos columnas que no ofrecen datos al análisis.

In [21]:
data.drop(lista_a_dropear, axis=1, inplace=True)

In [22]:
np.shape(data)

(3876, 53)

Estandarizamos las ids de las temporadas según la tabla de dimensión season.

In [23]:
unique_values = data['season_id'].unique()
unique_values

array([22020, 42020, 32020, 22021, 12021, 42021, 32021, 22022, 12022,
       42022, 32022])

In [24]:
replacement_dict = {
    22020: 2021, 42020: 2021, 32020: 2021,
    22021: 2022, 12021: 2022, 42021: 2022, 32021: 2022,
    22022: 2023, 12022: 2023, 42022: 2023, 32022: 2023
}


data['season_id'] = data['season_id'].replace(replacement_dict)

In [25]:
unique_values = data['season_id'].unique()
unique_values

array([2021, 2022, 2023])

In [26]:
data

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,game_minutes,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,season_type
61772,2021,1610612764,WAS,Washington Wizards,22000063,2020-12-31,WAS vs. CHI,L,240,42.0,84.0,0.500,14.0,29.0,0.483,32.0,41.0,0.780,9.0,28.0,37.0,27.0,8.0,4.0,14.0,27.0,130.0,-3,1610612741,CHI,Chicago Bulls,CHI @ WAS,W,50.0,92.0,0.543,14.0,36.0,0.389,19.0,25.0,0.760,9.0,35.0,44.0,34.0,9.0,3.0,17.0,27.0,133.0,3,Regular Season
61773,2021,1610612762,UTA,Utah Jazz,22000068,2020-12-31,UTA vs. PHX,L,240,35.0,84.0,0.417,12.0,34.0,0.353,13.0,19.0,0.684,10.0,36.0,46.0,16.0,8.0,3.0,14.0,15.0,95.0,-11,1610612756,PHX,Phoenix Suns,PHX @ UTA,W,41.0,84.0,0.488,17.0,33.0,0.515,7.0,10.0,0.700,5.0,36.0,41.0,26.0,5.0,6.0,15.0,17.0,106.0,11,Regular Season
61774,2021,1610612745,HOU,Houston Rockets,22000065,2020-12-31,HOU vs. SAC,W,240,45.0,86.0,0.523,13.0,36.0,0.361,19.0,25.0,0.760,10.0,35.0,45.0,26.0,10.0,6.0,18.0,14.0,122.0,3,1610612758,SAC,Sacramento Kings,SAC @ HOU,L,45.0,96.0,0.469,11.0,33.0,0.333,18.0,24.0,0.750,14.0,31.0,45.0,23.0,11.0,5.0,15.0,21.0,119.0,-3,Regular Season
61775,2021,1610612753,ORL,Orlando Magic,22000064,2020-12-31,ORL vs. PHI,L,240,33.0,95.0,0.347,7.0,28.0,0.250,19.0,21.0,0.905,10.0,38.0,48.0,17.0,2.0,5.0,11.0,15.0,92.0,-24,1610612755,PHI,Philadelphia 76ers,PHI @ ORL,W,44.0,91.0,0.484,15.0,33.0,0.455,13.0,15.0,0.867,7.0,47.0,54.0,23.0,7.0,10.0,15.0,23.0,116.0,24,Regular Season
61776,2021,1610612760,OKC,Oklahoma City Thunder,22000067,2020-12-31,OKC vs. NOP,L,240,30.0,80.0,0.375,15.0,48.0,0.313,5.0,15.0,0.333,7.0,33.0,40.0,24.0,6.0,3.0,19.0,20.0,80.0,-33,1610612740,NOP,New Orleans Pelicans,NOP @ OKC,W,44.0,91.0,0.484,13.0,36.0,0.361,12.0,19.0,0.632,14.0,44.0,58.0,23.0,9.0,4.0,13.0,16.0,113.0,33,Regular Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65692,2023,1610612743,DEN,Denver Nuggets,42200402,2023-06-04,DEN vs. MIA,L,240,39.0,75.0,0.520,11.0,28.0,0.393,19.0,22.0,0.864,9.0,29.0,38.0,23.0,7.0,2.0,14.0,21.0,108.0,-3,1610612748,MIA,Miami Heat,MIA @ DEN,W,38.0,78.0,0.487,17.0,35.0,0.486,18.0,20.0,0.900,8.0,23.0,31.0,28.0,5.0,4.0,11.0,22.0,111.0,3,Playoffs
65693,2023,1610612748,MIA,Miami Heat,42200403,2023-06-07,MIA vs. DEN,L,240,34.0,92.0,0.370,11.0,35.0,0.314,15.0,19.0,0.789,10.0,23.0,33.0,20.0,7.0,3.0,4.0,22.0,94.0,-15,1610612743,DEN,Denver Nuggets,DEN @ MIA,W,41.0,80.0,0.513,5.0,18.0,0.278,22.0,27.0,0.815,13.0,45.0,58.0,28.0,3.0,5.0,14.0,18.0,109.0,15,Playoffs
65694,2023,1610612748,MIA,Miami Heat,42200404,2023-06-09,MIA vs. DEN,L,240,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,8.0,29.0,37.0,23.0,2.0,3.0,15.0,19.0,95.0,-13,1610612743,DEN,Denver Nuggets,DEN @ MIA,W,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,0.762,5.0,29.0,34.0,26.0,11.0,7.0,8.0,18.0,108.0,13,Playoffs
65695,2023,1610612743,DEN,Denver Nuggets,42200405,2023-06-12,DEN vs. MIA,W,240,38.0,84.0,0.452,5.0,28.0,0.179,13.0,23.0,0.565,11.0,46.0,57.0,21.0,6.0,7.0,15.0,13.0,94.0,5,1610612748,MIA,Miami Heat,MIA @ DEN,L,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,0.875,11.0,33.0,44.0,18.0,9.0,7.0,8.0,21.0,89.0,-5,Playoffs


In [27]:
data.dtypes

season_id                          int64
team_id_home                       int64
team_abbreviation_home            object
team_name_home                    object
game_id                            int64
game_date                 datetime64[ns]
matchup_home                      object
wl_home                           object
game_minutes                       int64
fgm_home                         float64
fga_home                         float64
fg_pct_home                      float64
fg3m_home                        float64
fg3a_home                        float64
fg3_pct_home                     float64
ftm_home                         float64
fta_home                         float64
ft_pct_home                      float64
oreb_home                        float64
dreb_home                        float64
reb_home                         float64
ast_home                         float64
stl_home                         float64
blk_home                         float64
tov_home        

Verificamos datos duplicados.

In [28]:
duplicate_indices = data[data.duplicated(subset=['season_id', 'game_id', 'team_id_home', 'team_id_away'], keep=False)].index.tolist()

data.loc[duplicate_indices]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,game_minutes,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,season_type
62926,2021,1610616833,DRT,Team Durant,32000001,2021-03-07,DRT vs. LBN,L,240,60.0,114.0,0.526,27.0,72.0,0.375,3.0,5.0,0.6,10.0,31.0,41.0,47.0,15.0,0.0,14.0,8.0,150.0,-20,1610616834,LBN,Team LeBron,LBN @ DRT,W,68.0,107.0,0.636,31.0,61.0,0.508,3.0,5.0,0.6,8.0,40.0,48.0,46.0,9.0,3.0,18.0,8.0,170.0,20,All-Star
62927,2021,1610616833,DRT,Team Durant,32000001,2021-03-07,DRT vs. LBN,L,240,60.0,114.0,0.526,27.0,72.0,0.375,3.0,5.0,0.6,10.0,31.0,41.0,47.0,15.0,0.0,14.0,8.0,150.0,-20,1610616834,LBN,Team LeBron,LBN @ DRT,W,68.0,107.0,0.636,31.0,61.0,0.508,3.0,5.0,0.6,8.0,40.0,48.0,46.0,9.0,3.0,18.0,8.0,170.0,20,All Star


In [29]:
# Eliminar los duplicados en el mismo DataFrame
data.drop_duplicates(subset=['season_id', 'game_id', 'team_id_home', 'team_id_away'], keep='first', inplace=True)

In [30]:
duplicate_indices = data[data.duplicated(subset=['season_id', 'game_id', 'team_id_home', 'team_id_away'], keep=False)].index.tolist()

data.loc[duplicate_indices]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,game_minutes,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,season_type


In [31]:
np.shape(data)

(3875, 53)

## Ahora verificaremos la integridad de la tabla de dimensión game_info

In [32]:
#CARGA DEL DATASET
dim = pd.read_csv("game_info_filtrado.csv", sep=',')

In [33]:
# Encontrar las game_id que están en df_fact pero no en df_dim
missing_in_dim = set(data['game_id']) - set(dim['game_id'])

# Imprimir las game_id que faltan en df_dim
print('game_id presentes en df_fact pero faltan en df_dim:')
for game_id in missing_in_dim:
    print(game_id)

game_id presentes en df_fact pero faltan en df_dim:
22200322
22200333
42000401
22200357
22200360
22200376
22200382
22200383
22200387
12200015
22200400
12200019
22200408
12200028
22200414
22100064
22200419
12200037
22100072
22200435
12200061
22100093
22100096
12200065
22200455
22200459
22100110
22200473
22100123
22200477
22200482
22200483
22200485
22200495
22100149
22100150
22200505
22200506
22200511
22100162
22100164
22200517
22200530
22100178
22100179
22100183
22200538
22200540
22200557
22200560
22100214
22200567
22200568
22100220
22200576
22100227
22200583
22200584
22100231
22100234
22100237
22100238
22200596
22100248
22100250
22200618
22100268
22200634
22100284
22100285
22200642
22200647
22200652
22100302
22200661
22100321
22200675
22100323
22200677
22200678
22100334
22200690
22100338
22200695
22100355
22100369
22200723
22200733
22100383
22100396
22100399
22100402
22100403
12100020
22100407
22100414
12100033
22000066
22100422
22200779
22100427
22100430
12100047
22200784
22200785
222

### Observamos que faltan IDS en la tabla de dimensión. Procedemos a arreglarlo

In [34]:
data.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,game_minutes,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,pts_home,plus_minus_home,team_id_away,team_abbreviation_away,team_name_away,matchup_away,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,season_type
61772,2021,1610612764,WAS,Washington Wizards,22000063,2020-12-31,WAS vs. CHI,L,240,42.0,84.0,0.500,14.0,29.0,0.483,32.0,41.0,0.780,9.0,28.0,37.0,27.0,8.0,4.0,14.0,27.0,130.0,-3,1610612741,CHI,Chicago Bulls,CHI @ WAS,W,50.0,92.0,0.543,14.0,36.0,0.389,19.0,25.0,0.760,9.0,35.0,44.0,34.0,9.0,3.0,17.0,27.0,133.0,3,Regular Season
61773,2021,1610612762,UTA,Utah Jazz,22000068,2020-12-31,UTA vs. PHX,L,240,35.0,84.0,0.417,12.0,34.0,0.353,13.0,19.0,0.684,10.0,36.0,46.0,16.0,8.0,3.0,14.0,15.0,95.0,-11,1610612756,PHX,Phoenix Suns,PHX @ UTA,W,41.0,84.0,0.488,17.0,33.0,0.515,7.0,10.0,0.700,5.0,36.0,41.0,26.0,5.0,6.0,15.0,17.0,106.0,11,Regular Season
61774,2021,1610612745,HOU,Houston Rockets,22000065,2020-12-31,HOU vs. SAC,W,240,45.0,86.0,0.523,13.0,36.0,0.361,19.0,25.0,0.760,10.0,35.0,45.0,26.0,10.0,6.0,18.0,14.0,122.0,3,1610612758,SAC,Sacramento Kings,SAC @ HOU,L,45.0,96.0,0.469,11.0,33.0,0.333,18.0,24.0,0.750,14.0,31.0,45.0,23.0,11.0,5.0,15.0,21.0,119.0,-3,Regular Season
61775,2021,1610612753,ORL,Orlando Magic,22000064,2020-12-31,ORL vs. PHI,L,240,33.0,95.0,0.347,7.0,28.0,0.250,19.0,21.0,0.905,10.0,38.0,48.0,17.0,2.0,5.0,11.0,15.0,92.0,-24,1610612755,PHI,Philadelphia 76ers,PHI @ ORL,W,44.0,91.0,0.484,15.0,33.0,0.455,13.0,15.0,0.867,7.0,47.0,54.0,23.0,7.0,10.0,15.0,23.0,116.0,24,Regular Season
61776,2021,1610612760,OKC,Oklahoma City Thunder,22000067,2020-12-31,OKC vs. NOP,L,240,30.0,80.0,0.375,15.0,48.0,0.313,5.0,15.0,0.333,7.0,33.0,40.0,24.0,6.0,3.0,19.0,20.0,80.0,-33,1610612740,NOP,New Orleans Pelicans,NOP @ OKC,W,44.0,91.0,0.484,13.0,36.0,0.361,12.0,19.0,0.632,14.0,44.0,58.0,23.0,9.0,4.0,13.0,16.0,113.0,33,Regular Season


In [35]:
# Seleccionamos solo las columnas 4 y 5 (considerando que los índices empiezan en 0)
df_seleccionado = data.iloc[:, [4, 5]]

# Mostrar el DataFrame resultante
print(df_seleccionado)

        game_id  game_date
61772  22000063 2020-12-31
61773  22000068 2020-12-31
61774  22000065 2020-12-31
61775  22000064 2020-12-31
61776  22000067 2020-12-31
...         ...        ...
65692  42200402 2023-06-04
65693  42200403 2023-06-07
65694  42200404 2023-06-09
65695  42200405 2023-06-12
65696  32200001 2023-02-19

[3875 rows x 2 columns]


In [36]:
dim.columns

Index(['game_id', 'game_date', 'attendance'], dtype='object')

In [37]:
dim_final = dim.iloc[:, [0, 2]]

# Mostrar el DataFrame resultante
print(df_seleccionado)

        game_id  game_date
61772  22000063 2020-12-31
61773  22000068 2020-12-31
61774  22000065 2020-12-31
61775  22000064 2020-12-31
61776  22000067 2020-12-31
...         ...        ...
65692  42200402 2023-06-04
65693  42200403 2023-06-07
65694  42200404 2023-06-09
65695  42200405 2023-06-12
65696  32200001 2023-02-19

[3875 rows x 2 columns]


In [38]:
# Realiza el merge en base a la columna 'game_id'
df_combined = pd.merge(df_seleccionado, dim_final, on='game_id', how='left')

In [39]:
df_combined

,game_id,game_date,attendance
0,22000063,2020-12-31,15353.0
1,22000068,2020-12-31,15353.0
2,22000065,2020-12-31,15353.0
3,22000064,2020-12-31,15353.0
4,22000067,2020-12-31,15353.0
...,...,...,...
3870,42200402,2023-06-04,19537.0
3871,42200403,2023-06-07,20019.0
3872,42200404,2023-06-09,20184.0
3873,42200405,2023-06-12,19537.0


In [40]:
# Encuentra los game_id en df_fact que no están en df_dim
missing_game_ids = df_seleccionado[~df_seleccionado['game_id'].isin(df_combined['game_id'])]['game_id'].unique()

if len(missing_game_ids) > 0:
    print("Estos game_id están en df_fact pero no en df_dim:")
    print(missing_game_ids)
else:
    print("Todos los game_id en df_fact están presentes en df_dim.")

Todos los game_id en df_fact están presentes en df_dim.


In [41]:
# PORCENTAJE DE VALORES NULOS
nulos_total = df_combined.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0          0.000000
game_date               0          0.000000
attendance            349          9.006452


In [42]:
df_combined['attendance'] = df_combined['attendance'].fillna(np.median)

In [43]:
# PORCENTAJE DE VALORES NULOS
nulos_total = df_combined.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0               0.0
game_date               0               0.0
attendance              0               0.0


In [44]:
df_combined.to_csv('game_info_filtrado.csv', index=False)

### Ahora solo falta verificar que no hay IDS duplicadas en la tabla de dimensión

In [45]:
duplicados = df_combined.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = df_combined[duplicados]
print(datos_duplicados)

Empty DataFrame
Columns: [game_id, game_date, attendance]
Index: []


## Exportamos el dataset ya procesado.

In [46]:
data.to_csv('game_filtrado.csv', index=False)